In [11]:
import tensorflow as tf
from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np
import Utils
import keras
%matplotlib inline

In [12]:
vgg16 = keras.applications.VGG16(include_top=False,weights='imagenet')
vgg16_graph = keras.backend.get_session().graph

In [13]:
w_1_1 = vgg16_graph.get_tensor_by_name('block1_conv1/kernel:0')
b_1_1 = vgg16_graph.get_tensor_by_name('block1_conv1/bias:0')
w_1_2 = vgg16_graph.get_tensor_by_name('block1_conv2/kernel:0')
b_1_2 = vgg16_graph.get_tensor_by_name('block1_conv2/bias:0')

w_2_1 = vgg16_graph.get_tensor_by_name('block2_conv1/kernel:0')
b_2_1 = vgg16_graph.get_tensor_by_name('block2_conv1/bias:0')
w_2_2 = vgg16_graph.get_tensor_by_name('block2_conv2/kernel:0')
b_2_2 = vgg16_graph.get_tensor_by_name('block2_conv2/bias:0')

w_3_1 = vgg16_graph.get_tensor_by_name('block3_conv1/kernel:0')
b_3_1 = vgg16_graph.get_tensor_by_name('block3_conv1/bias:0')
w_3_2 = vgg16_graph.get_tensor_by_name('block3_conv2/kernel:0')
b_3_2 = vgg16_graph.get_tensor_by_name('block3_conv2/bias:0')
w_3_3 = vgg16_graph.get_tensor_by_name('block3_conv3/kernel:0')
b_3_3 = vgg16_graph.get_tensor_by_name('block3_conv3/bias:0')

w_4_1 = vgg16_graph.get_tensor_by_name('block4_conv1/kernel:0')
b_4_1 = vgg16_graph.get_tensor_by_name('block4_conv1/bias:0')
w_4_2 = vgg16_graph.get_tensor_by_name('block4_conv2/kernel:0')
b_4_2 = vgg16_graph.get_tensor_by_name('block4_conv2/bias:0')
w_4_3 = vgg16_graph.get_tensor_by_name('block4_conv3/kernel:0')
b_4_3 = vgg16_graph.get_tensor_by_name('block4_conv3/bias:0')

w_5_1 = vgg16_graph.get_tensor_by_name('block5_conv1/kernel:0')
b_5_1 = vgg16_graph.get_tensor_by_name('block5_conv1/bias:0')
w_5_2 = vgg16_graph.get_tensor_by_name('block5_conv2/kernel:0')
b_5_2 = vgg16_graph.get_tensor_by_name('block5_conv2/bias:0')
w_5_3 = vgg16_graph.get_tensor_by_name('block5_conv3/kernel:0')
b_5_3 = vgg16_graph.get_tensor_by_name('block5_conv3/bias:0')

In [16]:
def unpool_with_argmax(pool, ind, name = None, ksize=[1, 2, 2, 1]):

    """
       Unpooling layer after max_pool_with_argmax.
       Args:
           pool:   max pooled output tensor
           ind:      argmax indices
           ksize:     ksize is the same as for the pool
       Return:
           unpool:    unpooling tensor
    """
    with tf.variable_scope(name):
        input_shape = pool.get_shape().as_list()
        print("input_shape: {0}, ind: {1}".format(pool.shape, ind.shape))
        output_shape = (input_shape[0], input_shape[1] * ksize[1], input_shape[2] * ksize[2], input_shape[3])

        flat_input_size = tf.reduce_prod(input_shape)
        flat_output_shape = [output_shape[0], output_shape[1] * output_shape[2] * output_shape[3]]

        pool_ = tf.reshape(pool, [flat_input_size])
        batch_range = tf.reshape(tf.range(output_shape[0], dtype=ind.dtype), shape=[input_shape[0], 1, 1, 1])
        b = tf.ones_like(ind) * batch_range
        b = tf.reshape(b, [flat_input_size, 1])
        ind_ = tf.reshape(ind, [flat_input_size, 1])
        ind_ = tf.concat([b, ind_], 1)

        ret = tf.scatter_nd(ind_, pool_, shape=flat_output_shape)
        ret = tf.reshape(ret, output_shape)
        return ret

In [17]:
conv_1_1 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = images,filter=w_1_1, strides=strides, padding=padding), b_1_1)))
conv_1_2 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = conv_1_1,filter=w_1_2, strides=strides, padding=padding), b_1_2)))
max_pooling_1, indicies_1 = tf.nn.max_pool_with_argmax(conv_1_2, ksize=max_pooling_ksize,strides=(1,2,2,1), padding='VALID')

conv_2_1 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = max_pooling_1,filter=w_2_1, strides=strides, padding=padding), b_2_1)))
conv_2_2 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = conv_2_1,filter=w_2_2, strides=strides, padding=padding), b_2_2)))
max_pooling_2, indicies_2 = tf.nn.max_pool_with_argmax(conv_2_2, ksize=max_pooling_ksize,strides=(1,2,2,1), padding='VALID')

conv_3_1 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = max_pooling_2,filter=w_3_1, strides=strides, padding=padding), b_3_1)))
conv_3_2 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = conv_3_1,filter=w_3_2, strides=strides, padding=padding), b_3_2)))
conv_3_3 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = conv_3_2,filter=w_3_3, strides=strides, padding=padding), b_3_3)))
max_pooling_3, indicies_3 = tf.nn.max_pool_with_argmax(conv_3_3, ksize=max_pooling_ksize,strides=(1,2,2,1), padding='VALID')

conv_4_1 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = max_pooling_3,filter=w_4_1, strides=strides, padding=padding), b_4_1)))
conv_4_2 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = conv_4_1,filter=w_4_2, strides=strides, padding=padding), b_4_2)))
conv_4_3 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = conv_4_2,filter=w_4_3, strides=strides, padding=padding), b_4_3)))
max_pooling_4, indicies_4 = tf.nn.max_pool_with_argmax(conv_4_3, ksize=max_pooling_ksize,strides=(1,2,2,1), padding='VALID')

conv_5_1 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = max_pooling_4,filter=w_5_1, strides=strides, padding=padding), b_5_1)))
conv_5_2 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = conv_5_1,filter=w_5_2, strides=strides, padding=padding), b_5_2)))
conv_5_3 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = conv_5_2,filter=w_5_3, strides=strides, padding=padding), b_5_3)))
#max_pooling_5, indicies_5 = tf.nn.max_pool_with_argmax(conv_5_3, ksize=max_pooling_ksize,strides=(1,2,2,1), padding='VALID', Targmax=tf.int32)
max_pooling_5, indicies_5 = tf.nn.max_pool_with_argmax(conv_5_3, ksize=max_pooling_ksize,strides=(1,2,2,1), padding='VALID')

print("max_pooling_5: {0}".format(max_pooling_5.shape))
upsample_1 = unpool_with_argmax(max_pooling_5, indicies_5, name='upsample_1')
print("upsample_1: {0}".format(upsample_1.shape))
deconv_1_1 = tf.layers.batch_normalization(tf.layers.conv2d(inputs=upsample_1, filters=w_5_3.shape[2].value,kernel_size=(3,3),activation=tf.nn.relu, name='deconv_1_1',padding='same'))
print("deconv_1_1: {0}".format(deconv_1_1.shape))
deconv_1_2 = tf.layers.batch_normalization(tf.layers.conv2d(deconv_1_1, w_5_2.shape[2].value,(3,3),activation=tf.nn.relu, name='deconv_1_2',padding='same'))
print("deconv_1_2: {0}".format(deconv_1_2.shape))
deconv_1_3 = tf.layers.batch_normalization(tf.layers.conv2d(deconv_1_2, w_5_1.shape[2].value,(3,3),activation=tf.nn.relu, name='deconv_1_3',padding='same'))
print("deconv_1_3: {0}".format(deconv_1_3.shape))

upsample_2 = unpool_with_argmax(deconv_1_3, indicies_4, name='upsample_2')
deconv_2_1 = tf.layers.batch_normalization(tf.layers.conv2d(upsample_2, w_4_3.shape[2].value,(3,3),activation=tf.nn.relu, name='deconv_2_1',padding='same'))
deconv_2_2 = tf.layers.batch_normalization(tf.layers.conv2d(deconv_2_1, w_4_2.shape[2].value,(3,3),activation=tf.nn.relu, name='deconv_2_2',padding='same'))
deconv_2_3 = tf.layers.batch_normalization(tf.layers.conv2d(deconv_2_2, w_4_1.shape[2].value,(3,3),activation=tf.nn.relu, name='deconv_2_3',padding='same'))

upsample_3 = unpool_with_argmax(deconv_2_3, indicies_3, name='upsample_3')
deconv_3_1 = tf.layers.batch_normalization(tf.layers.conv2d(upsample_3, w_3_3.shape[2].value,(3,3),activation=tf.nn.relu, name='deconv_3_1',padding='same'))
deconv_3_2 = tf.layers.batch_normalization(tf.layers.conv2d(deconv_3_1, w_3_2.shape[2].value,(3,3),activation=tf.nn.relu, name='deconv_3_2',padding='same'))
deconv_3_3 = tf.layers.batch_normalization(tf.layers.conv2d(deconv_3_2, w_3_1.shape[2].value,(3,3),activation=tf.nn.relu, name='deconv_3_3',padding='same'))

upsample_4 = unpool_with_argmax(deconv_3_3, indicies_2, name='upsample_4')
deconv_4_1 = tf.layers.batch_normalization(tf.layers.conv2d(upsample_4, w_2_2.shape[2].value,(3,3),activation=tf.nn.relu, name='deconv_4_1',padding='same'))
deconv_4_2 = tf.layers.batch_normalization(tf.layers.conv2d(deconv_4_1, w_2_1.shape[2].value,(3,3),activation=tf.nn.relu, name='deconv_4_2',padding='same'))

upsample_5 = unpool_with_argmax(deconv_4_2, indicies_1, name='upsample_5')
deconv_5_1 = tf.layers.batch_normalization(tf.layers.conv2d(upsample_5, w_1_2.shape[2].value,(3,3),activation=tf.nn.relu, name='deconv_5_1',padding='same'))
#deconv_5_2 = tf.layers.conv2d(deconv_5_1, w_1_1.shape[2].value,(3,3),activation=tf.nn.relu, name='deconv_5_2',padding='same')
deconv_5_2 = tf.layers.batch_normalization(tf.layers.conv2d(deconv_5_1,number_of_classes,(3,3),activation=tf.nn.relu, name='deconv_5_2',padding='same'))
print("deconv_5_2: {0}".format(deconv_5_2.shape))
out = tf.nn.softmax(deconv_5_2)
print("out: {0}".format(out.shape))
print("out: {0}".format(labels_im))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=labels_im))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correctPred = tf.equal(tf.argmax(out,-1), tf.argmax(labels_im,-1))
print("correctPred: {0}".format(correctPred.shape))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))
print("acurracy: {0}".format(accuracy.shape))

#reverseOneHot = tf.argmax(out,-1)
#reverseOneHot = tf.reshape(reverseOneHot,(reverseOneHot.shape[1].value,reverseOneHot.shape[2].value))
#print("reverseOneHot: {0}".format(reverseOneHot.shape))


max_pooling_5: (4, 11, 15, 512)
input_shape: (4, 11, 15, 512), ind: (4, 11, 15, 512)
upsample_1: (4, 22, 30, 512)


ValueError: Variable deconv_1_1/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\Daniel\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1470, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\Daniel\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2956, in create_op
    op_def=op_def)
  File "C:\Users\Daniel\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


In [15]:
number_of_classes = 12
batch_size = 4
input_shape = (batch_size, 352, 480, 3)
label_shape = (batch_size, 352, 480, number_of_classes)

images=tf.placeholder(dtype=tf.float32,shape=input_shape)
labels_im=tf.placeholder(dtype=tf.float32,shape=label_shape)

#images=tf.placeholder(dtype=tf.float32,shape=(1, 720, 960, 3))
strides = (1,1,1,1)
padding = 'SAME'
max_pooling_ksize = (1,2,2,1)
learning_rate = 0.0001
num_epochs = 10

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
train_images = np.asarray(Utils.load_images("../Data/CamVid/train/"))
test_images = np.asarray(Utils.load_images("../Data/CamVid/test/"))

In [ ]:
train_annotations = np.asarray(Utils.load_annotations("../Data/CamVid/trainannot/"))
test_annotations = np.asarray(Utils.load_annotations("../Data/CamVid/testannot/"))

In [ ]:
training_bp = Utils.BatchProcessor()
test_bp = Utils.BatchProcessor()

In [ ]:
for i in range(int(num_epochs*len(train_images) / batch_size)):
    batch_X, batch_y = training_bp.get_next_batch(train_images, train_annotations, batch_size)
    
    sess.run(train_step,feed_dict={images: batch_X,labels_im: batch_y})
    
    if(i%50 == 0):
        accus = []
        losss = []
        for j in range(int(len(test_images)/batch_size)):
            test_batch_x, test_batch_y = test_bp.get_next_batch(test_images, test_annotations, batch_size)
            loss_o, accu_o = sess.run([loss,accuracy],feed_dict={images: test_batch_x,labels_im: test_batch_y})
            accus.append(accu_o)
            losss.append(loss_o)
            
        print("Test accu: {0}, test loss: {1}".format(np.mean(accus), np.mean(losss)))


In [ ]:
import time

start_time = time.time()

sess.run(max_pooling_5,feed_dict={images: image})

print("--- %s seconds ---" % (time.time() - start_time))

loss_sum,acc_sum = sess.run([loss,accuracy],feed_dict={images: image,labels_im: label_image})
print("without training:")
print("Loss:",loss_sum)
print("Accuracy:",acc_sum)


start_time = time.time()
for i in range(0,100):
    sess.run(train_step,feed_dict={images: image,labels_im: label_image})
    loss_sum,acc_sum = sess.run([loss,accuracy],feed_dict={images: image,labels_im: label_image})
    if i % 10 == 0:
        print("Loss:",loss_sum)
        print("Accuracy:",acc_sum)
        pass

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
roh = sess.run(reverseOneHot,feed_dict={images: image,labels_im: label_image})

plt.imshow(roh)


In [ ]:
roh

In [ ]:
im = imread('../Data/CamVid/trainannot/0001TP_006690.png')
im = im[0:192, 0:256]
plt.imshow(im)